In [9]:
import pandas as pd
file_path = 'Stud_E-mobility_data_staticLimit.csv'
data = pd.read_csv(file_path)
data.head()


,_time,GARAGE_EXTERNAL_POWER,DEMAND_LIMIT,DEMAND_LIMIT_INDICATOR,BATTERY_SOC,BATTERY_DISCHARGE_POWER,BATTERY_CHARGED_ENERGY,BATTERY_DISCHARGED_ENERGY,PV_POWER,PV_ENERGY,...,WALLBOX_3_ENERGY,WALLBOX_3_POWER,WALLBOX_A_ENERGY,WALLBOX_A_POWER,WALLBOX_B_ENERGY,WALLBOX_B_POWER,WALLBOX_C_ENERGY,WALLBOX_C_POWER,WALLBOX_FASTCHARGER_ENERGY,WALLBOX_FASTCHARGER_POWER
0,2023-10-05 00:01:00+00:00,1.244171,50.0,1.0,40.5,-0.338,0.0,0.0,0.008693,0.0,...,0.0,0.006914,0.000000,0.010233,0.000000,0.010435,0.0,0.010112,0.000000,0.154965
1,2023-10-05 00:02:00+00:00,1.244171,50.0,1.0,40.5,-0.372,0.0,0.0,0.008693,0.0,...,0.0,0.007550,0.000977,0.010233,0.000000,0.010435,0.0,0.010112,0.007812,0.155586
2,2023-10-05 00:03:00+00:00,1.244171,50.0,1.0,40.5,-0.393,0.0,0.0,0.008693,0.0,...,0.0,0.006996,0.000000,0.010233,0.000000,0.010435,0.0,0.010112,0.000000,0.157340
3,2023-10-05 00:04:00+00:00,1.244171,50.0,1.0,40.5,-0.339,0.0,0.0,0.008693,0.0,...,0.0,0.007392,0.000000,0.010233,0.000977,0.010435,0.0,0.010112,0.000000,0.158009
4,2023-10-05 00:05:00+00:00,1.244171,50.0,1.0,40.5,-0.371,0.0,0.0,0.008693,0.0,...,0.0,0.007188,0.000000,0.010233,0.000000,0.010435,0.0,0.010112,0.007812,0.158804


In [2]:

# Initialize new columns for adjusted values
data['BATTERY_DISCHARGE_POWER_ADJUSTED'] = 0
data['BATTERY_CHARGE_POWER_ADJUSTED'] = 0
# Conditions based on SOC
cond_high_soc = data['BATTERY_SOC'] > 80
cond_mid_soc = (data['BATTERY_SOC'] > 40) & (data['BATTERY_SOC'] <= 80)
cond_low_soc = (data['BATTERY_SOC'] <= 40) & (data['BATTERY_SOC'] >= 15)
cond_critical_soc = data['BATTERY_SOC'] < 15
# External power conditions
cond_above_limit = data['GARAGE_EXTERNAL_POWER'] > data['DEMAND_LIMIT']
cond_below_limit = data['GARAGE_EXTERNAL_POWER'] < data['DEMAND_LIMIT']
# Apply rules for high SOC (>80%)
data.loc[cond_high_soc, 'BATTERY_DISCHARGE_POWER_ADJUSTED'] = data['GARAGE_EXTERNAL_POWER']
# Apply rules for mid SOC (40% < SOC <= 80%)
data.loc[cond_mid_soc & cond_above_limit, 'BATTERY_DISCHARGE_POWER_ADJUSTED'] = data['GARAGE_EXTERNAL_POWER'] - data['DEMAND_LIMIT']
# Apply rules for low SOC (SOC < 40%)
# Use battery if above limit
data.loc[cond_low_soc & cond_above_limit, 'BATTERY_DISCHARGE_POWER_ADJUSTED'] = data['GARAGE_EXTERNAL_POWER'] - data['DEMAND_LIMIT']
# Charge battery if below limit
data.loc[cond_low_soc & cond_below_limit, 'BATTERY_CHARGE_POWER_ADJUSTED'] = data['DEMAND_LIMIT'] - data['GARAGE_EXTERNAL_POWER']
# Apply rules for critical SOC (<15%)
data.loc[cond_critical_soc, 'BATTERY_DISCHARGE_POWER_ADJUSTED'] = 0  # Stop discharging
# Display the first few rows of the adjusted dataset to verify changes
data[['BATTERY_SOC', 'GARAGE_EXTERNAL_POWER', 'DEMAND_LIMIT', 'BATTERY_DISCHARGE_POWER_ADJUSTED', 'BATTERY_CHARGE_POWER_ADJUSTED']].head()


,BATTERY_SOC,GARAGE_EXTERNAL_POWER,DEMAND_LIMIT,BATTERY_DISCHARGE_POWER_ADJUSTED,BATTERY_CHARGE_POWER_ADJUSTED
0,40.5,1.244171,50.0,0.0,0.0
1,40.5,1.244171,50.0,0.0,0.0
2,40.5,1.244171,50.0,0.0,0.0
3,40.5,1.244171,50.0,0.0,0.0
4,40.5,1.244171,50.0,0.0,0.0


In [3]:
# Save the adjusted dataset to a new CSV file
adjusted_file_path = 'Adjusted_E-mobility_data_staticLimit.csv'
data.to_csv(adjusted_file_path, index=False)

adjusted_file_path


'Adjusted_E-mobility_data_staticLimit.csv'

In [5]:
!pip install tensorflow
!pip install logictensornetworks


ERROR: Could not find a version that satisfies the requirement logictensornetworks (from versions: none)
ERROR: No matching distribution found for logictensornetworks


In [6]:
!pip install ltn

### Proposal Predicates:
1. High SOC (SOC > 80%)

2. Mid SOC (40% < SOC < 80%)

3. Low SOC (SOC < 40%)

4. Critical SOC (SOC < 15%)
5. Above Demand Limit (GARAGE_EXTERNAL_POWER > DEMAND_LIMIT)

6. Below Demand Limit (GARAGE_EXTERNAL_POWER < DEMAND_LIMIT)


For High SOC: All e-car charging should be covered by the local battery.


For Mid SOC: If power from the grid exceeds the demand limit, e-car charging should be covered by the battery.


For Low SOC: Similar to Mid SOC, but includes conditions for charging the battery from the grid if under the demand limit.

For Critical SOC: Discharge is halted to preserve battery health.

1. Let's use us

In [ ]:
import tensorflow as tf
import logictensornetworks as ltn
# Data placeholders
SOC = tf.placeholder(tf.float32, shape=[None, 1])
GARAGE_EXTERNAL_POWER = tf.placeholder(tf.float32, shape=[None, 1])
DEMAND_LIMIT = tf.placeholder(tf.float32, shape=[None, 1])
# Predicate definitions
@ltn.Predicate.Lambda
def HighSOC(data):
    return data['SOC'] > 80
@ltn.Predicate.Lambda
def MidSOC(data):
    return (data['SOC'] > 40) & (data['SOC'] <= 80)
@ltn.Predicate.Lambda
def LowSOC(data):
    return (data['SOC'] <= 40)
@ltn.Predicate.Lambda
def CriticalSOC(data):
    return data['SOC'] < 15
@ltn.Predicate.Lambda
def AboveDemandLimit(data):
    return data['GARAGE_EXTERNAL_POWER'] > data['DEMAND_LIMIT']
@ltn.Predicate.Lambda
def BelowDemandLimit(data):
    return data['GARAGE_EXTERNAL_POWER'] < data['DEMAND_LIMIT']
all_high_soc_covered = ltn.Forall(SOC, HighSOC(SOC), p=1)  # Ensures all high SOC conditions meet the battery usage rule
mid_soc_demand_control = ltn.Forall(SOC, ltn.Implies(MidSOC(SOC) & AboveDemandLimit(SOC), HighSOC(SOC)), p=1)
low_soc_charge_or_discharge = ltn.Forall(SOC, ltn.And(LowSOC(SOC) & AboveDemandLimit(SOC), LowSOC(SOC) & BelowDemandLimit(SOC)), p=1)
stop_discharge_at_critical = ltn.Forall(SOC, ltn.Implies(CriticalSOC(SOC), ltn.Not(HighSOC(SOC))), p=1)
